In [2]:
import logging
log_level = logging.INFO
# log_level = logging.DEBUG
log_level = 15
logging.basicConfig(level=log_level)
logger = logging.getLogger(__name__)
logger.setLevel(log_level)

import os
from dotenv import load_dotenv
load_dotenv()
secret_name = "OPENAI_API_KEY_CEMAD"
openai_api_key = os.getenv(secret_name)
key = os.getenv('DECRYPTION_KEY_CEMAD')

import importlib
from openai import OpenAI
from regulations_rag.file_tools import load_parquet_data, save_parquet_data

import data_tools.index_tools
importlib.reload(data_tools.index_tools)
from data_tools.index_tools import update_text_in_index, add_to_index, remove_from_index

openai_client = OpenAI(api_key=openai_api_key,)
key = os.getenv('excon_encryption_key')

In [5]:
to_change = "index"
#to_change = "index_plus"
if to_change == "index":
    file = "./inputs/index/ad_index.parquet"
else:
    file = "./inputs/index/ad_index_plus.parquet"

df = load_parquet_data(file, key)


In [57]:
import sqlite3
import sqlite_vec
import pandas as pd

# Connect to SQLite and create the FTS table
conn = sqlite3.connect('./db/fts_database.db')
conn.enable_load_extension(True) # load the vector embedding extension
sqlite_vec.load(conn)
conn.enable_load_extension(False)
cursor = conn.cursor()


In [27]:
tmp = df.iloc[0]['embedding']
len(tmp)

1024

In [61]:

# Create the FTS5 table for full-text search on the `text` column
cursor.execute('''
    CREATE VIRTUAL TABLE IF NOT EXISTS document_paragraphs
    USING fts5(section_reference, text, source, document);
''')

# Insert the data from the DataFrame into the FTS table
for _, row in df.iterrows():
    cursor.execute('''
        INSERT INTO document_paragraphs (section_reference, text, source, document)
        VALUES (?, ?, ?, ?)
    ''', (row['section_reference'], row['text'], row['source'], row['document']))

conn.commit()



In [60]:
import numpy as np
# Optional: Store embeddings in a separate table linked by rowid
# https://github.com/asg017/sqlite-vec/blob/main/examples/simple-python/demo.py
conn.execute('CREATE VIRTUAL TABLE IF NOT EXISTS embeddings USING vec0(embedding float[1024])')

# Insert embeddings linked to the FTS table rowids
for idx, embedding in enumerate(df['embedding']):
    #print(type(embedding))
    cursor.execute('INSERT INTO embeddings(embedding) VALUES (?)',
                   ([embedding.astype(np.float32)]))
    # cursor.execute('INSERT INTO embeddings(rowid, embedding) VALUES (?, ?)',
    #                (idx + 1, [embedding.astype(np.float32)]))

conn.commit()


In [63]:
# Define the search query
terms = ["managing institution"]

# Join terms with ' AND ' for an AND query in FTS
search_query = ' AND '.join(terms)

# Perform the FTS search
cursor.execute('''
    SELECT section_reference, text, source, document
    FROM document_paragraphs
    WHERE document_paragraphs MATCH ?
''', (search_query,))

# Fetch and display results
results = cursor.fetchall()

# Display the results
for row in results:
    section_reference, text, source, document = row
    print(f"Section: {section_reference}")
    print(f"Text: {text}")
    print(f"Source: {source}")
    print(f"Document: {document}\n")



Section: B.2(H)(iii)
Text: Capital transfers. South African institutional investors. The distinction between managing institution and originating institution
Source: heading
Document: CEMAD

Section: B.2(H)(iii)
Text: A managing institution refers to a life insurer, CIS manager, or discretionary financial services provider registered with the Financial Surveillance Department. They offer investment products to both institutional and retail clients. On the other hand, an originating institution is an institutional investor that qualifies for a prudential limit and chooses to invest in products provided by a managing institution. This can be done directly or via intermediaries, like a discretionary financial services provider not registered as an institutional investor with the Financial Surveillance Department or an administrative financial services provider.
Source: summary
Document: CEMAD

Section: B.2(H)(v)
Text: Authorised Dealers, when transferring funds for institutional investors

In [55]:
conn.close()